# Add a Link: User Dataset

We're looking to analyze the Add a Link experiment, and therefore start by grabbing a canonical dataset of users who were part of the experiment. The phab task for this work is [T286816](https://phabricator.wikimedia.org/T286816)

We use the MediaWiki History dataset as the authoritative source of user registrations, then augment it with information from ServerSideAccountCreation and the MediaWiki databases. This process is similar to how we've done it previously for other experiments such as NEWTEA.

The experiment started on 2021-05-27 at 19:12:03 UTC, ref [T277356#7120922](https://phabricator.wikimedia.org/T277356#7120922) At the start, it was deployed to four Wikipedias: Arabic, Bengali, Czech, and Vietnamese

It was deployed to additional wikis on 2021-07-21 at 11:11:21 UTC, ref [T284481#7226968](https://phabricator.wikimedia.org/T284481#7226968) This added the following Wikipedias: Russian, French, Polish, Romanian, Persian, and Hungarian.

Users that get the Growth Features are split 50/50 between having Add a Link and having the unstructured link task.

In [2]:
import json
import datetime as dt

from collections import defaultdict

import numpy as np
import pandas as pd

from wmfdata import hive, spark, mariadb

In [14]:
## Deployment to the first group of Wikipedias, ref above
group_1_ts = dt.datetime(2021, 5, 27, 19, 12, 3)
group_1_wikis = ['arwiki', 'bnwiki', 'cswiki', 'viwiki']

## Deployment to the second group of Wikipedias, ref above
group_2_ts = dt.datetime(2021, 7, 21, 11, 11, 21)
group_2_wikis = ['fawiki', 'frwiki', 'huwiki', 'plwiki', 'rowiki', 'ruwiki']

## End of data gathering
## May 27 was a Thursday
## July 21 was a Wednesday
## Let's use a whole number of weeks from a Thursday as our end date
## We need 15 days of data past the end date. That gives us Oct 14 as the last day
end_ts = dt.datetime(2021, 10, 14, 0, 0, 0)

## The MediaWiki history snapshot we use for our data gathering
mwh_snapshot = '2021-10'

## Lists of known users to ignore (e.g. test accounts and experienced users)
known_users = defaultdict(set)
known_users['cswiki'].update([14, 127629, 303170, 342147, 349875, 44133, 100304, 307410, 439792, 444907,
                              454862, 456272, 454003, 454846, 92295, 387915, 398470, 416764, 44751, 132801,
                              137787, 138342, 268033, 275298, 317739, 320225, 328302, 339583, 341191,
                              357559, 392634, 398626, 404765, 420805, 429109, 443890, 448195, 448438,
                              453220, 453628, 453645, 453662, 453663, 453664, 440694, 427497, 272273,
                              458025, 458487, 458049, 59563, 118067, 188859, 191908, 314640, 390445,
                              451069, 459434, 460802, 460885, 79895, 448735, 453176, 467557, 467745,
                              468502, 468583, 468603, 474052, 475184, 475185, 475187, 475188, 294174,
                              402906, 298011])

known_users['kowiki'].update([303170, 342147, 349875, 189097, 362732, 384066, 416362, 38759, 495265,
                              515553, 537326, 566963, 567409, 416360, 414929, 470932, 472019, 485036,
                              532123, 558423, 571587, 575553, 576758, 360703, 561281, 595100, 595105,
                              595610, 596025, 596651, 596652, 596653, 596654, 596655, 596993, 942,
                              13810, 536529])

known_users['viwiki'].update([451842, 628512, 628513, 680081, 680083, 680084, 680085, 680086, 355424,
                              387563, 443216, 682713, 659235, 700934, 705406, 707272, 707303, 707681, 585762])

known_users['arwiki'].update([237660, 272774, 775023, 1175449, 1186377, 1506091, 1515147, 1538902,
                              1568858, 1681813, 1683215, 1699418, 1699419, 1699425, 1740419, 1759328, 1763990])

## Grab the user IDs of known test accounts so they can be added to the exclusion list

def get_known_users(wiki):
    '''
    Get user IDs of known test accounts and return a set of them.
    '''
    
    username_patterns = ["MMiller", "Zilant", "Roan", "KHarlan", "MWang", "SBtest",
                         "Cloud", "Rho2019", "Test"]

    known_user_query = '''
SELECT user_id
FROM user
WHERE user_name LIKE "{name_pattern}%"
    '''
    
    known_users = set()
    
    for u_pattern in username_patterns:
        new_known = mariadb.run(known_user_query.format(
            name_pattern = u_pattern), wiki)
        known_users = known_users | set(new_known['user_id'])

    return(known_users)
        
for wiki in group_1_wikis + group_2_wikis:
    known_users[wiki] = known_users[wiki] | get_known_users(wiki)

In [4]:
## Filename of where the canonical datasets are stored.
tsv_canonical_user_file = '/home/nettrom/src/2021-Growth-structured-tasks/datasets/add-a-link-experiment-canonical-users.tsv'
tsv_datalake_user_file = '/home/nettrom/src/2021-Growth-structured-tasks/datasets/add-a-link-experiment-datalake-users.tsv'
canonical_user_table = 'nettrom_growth.addalink_exp_users'

## Helper Functions

In [7]:
def make_known_users_sql(kd, wiki_column, user_column):
    '''
    Based on the dictionary `kd` mapping wiki names to sets of user IDs of known users,
    create a SQL expression to exclude users based on the name of the wiki matching `wiki_column`
    and the user ID not matching `user_column`
    '''
    
    wiki_exp = '''({w_column} = '{wiki}' AND {u_column} NOT IN ({id_list}))'''
    
    expressions = list()

    ## Iteratively build the expression for each wiki
    for wiki_name, wiki_users in kd.items():
        expressions.append(wiki_exp.format(
            w_column = wiki_column,
            wiki = wiki_name,
            u_column = user_column,
            id_list = ','.join([str(u) for u in wiki_users])
        ))
    
    ## We then join all the expressions with an OR, and we're done.
    return(' OR '.join(expressions))
    

In [19]:
def make_partition_statement(start_ts, end_ts, prefix = ''):
    '''
    This takes the two timestamps and creates a statement that selects
    partitions based on `year`, `month`, and `day` in order to make our
    data gathering not use excessive amounts of data. It assumes that
    `start_ts` and `end_ts` are either in the same year, or if spanning
    a year boundary are within a month apart.
    This assumption simplifies the code and output a lot.
    
    An optional prefix can be set to enable selecting partitions for
    multiple tables with different aliases.
    
    :param start_ts: start timestamp
    :type start_ts: datetime.datetime
    
    :param end_ts: end timestamp
    :type end_ts: datetime.datetime
    
    :param prefix: prefix to use in front of partition clauses, "." is added automatically
    :type prefix: str
    '''
    
    if prefix:
        prefix = f'{prefix}.' # adds "." after the prefix
    
    # there are three cases:
    # 1: month and year are the same, output a "BETWEEN" statement with the days
    # 2: the years are the same, and the months differ by 1: output a statement for each month
    # 3: the years are the same: create a list of statements from start_ts.month to end_ts.month,
    #    return them OR'ed together
    # 4: the years differ by 1, start_ts is December and end_ts is January, do the same as #2
    # 5: anything else, raise an exception because this isn't implemented yet.
    
    if start_ts.year == end_ts.year and start_ts.month == end_ts.month:
        return(f'''{prefix}year = {start_ts.year}
AND {prefix}month = {start_ts.month}
AND {prefix}day BETWEEN {start_ts.day} AND {end_ts.day}''')
    elif start_ts.year == end_ts.year and (end_ts.month - start_ts.month) == 1:
        return(f'''
(
    ({prefix}year = {start_ts.year}
     AND {prefix}month = {start_ts.month}
     AND {prefix}day >= {start_ts.day})
 OR ({prefix}year = {end_ts.year}
     AND {prefix}month = {end_ts.month}
     AND {prefix}day <= {end_ts.day})
)''')
    elif start_ts.year == end_ts.year:
        # do the start month as a list
        parts = [f'''({prefix}year = {start_ts.year}
     AND {prefix}month = {start_ts.month}
     AND {prefix}day >= {start_ts.day})''']
        # for month +1 to end month, add each month
        for m in range(start_ts.month+1, end_ts.month):
            parts.append(f'''({prefix}year = {start_ts.year}
            AND {prefix}month = {m})''')
        # then append the end month and return a parenthesis OR'ed together of all of it
        parts.append(f'''({prefix}year = {end_ts.year}
     AND {prefix}month = {end_ts.month}
     AND {prefix}day <= {end_ts.day})''')
        return('({})'.format(
            '\nOR\n'.join(parts)
        ))
    elif (end_ts.year - start_ts.year) == 1 and start_ts.month == 12 and end_ts.month == 1:
        return(f'''
(
    ({prefix}year = {start_ts.year}
     AND {prefix}month = {start_ts.month}
     AND {prefix}day >= {start_ts.day})
 OR ({prefix}year = {end_ts.year}
     AND {prefix}month = {end_ts.month}
     AND {prefix}day <= {end_ts.day})
)''')
    else:
        raise Exception('Difference between start and end timestamps is not implemented. See code for details.')


## User Registrations

We get user registrations for users registered between the deployment to a given group and the end of data gathering, separately for each group of wikis. Known users are excluded individually for each wiki based on `known_users`

In [9]:
## Query to get user registrations on the given wikis for the given dates, ignoring
## known users.

user_registrations_query = '''
SELECT
    wiki_db,
    user_id,
    user_registration_timestamp
FROM wmf.mediawiki_user_history
WHERE snapshot = "{snapshot}"
AND caused_by_event_type = "create" -- account registration
AND created_by_self = true -- no auto-created accounts
AND SIZE(is_bot_by) = 0 -- no bots
AND SIZE(is_bot_by_historical) = 0 -- definitely no bots
AND (
        (wiki_db IN ({group_1_wiki_list})
         AND user_registration_timestamp > "{group_1_start_timestamp}"
         AND user_registration_timestamp < "{end_timestamp}")
    OR
        (wiki_db IN ({group_2_wiki_list})
         AND user_registration_timestamp > "{group_2_start_timestamp}"
         AND user_registration_timestamp < "{end_timestamp}")
    )
AND ({known_user_id_expression})
'''

In [16]:
## NOTE: if you use growth.utils.hive_format to format the timestamps, then you miss the first day of data
## as the "T" in the Hive format isn't in the timestamps in mediawiki_history. And you probably get
## an extra day of data at the end.

user_registrations = spark.run(
    user_registrations_query.format(
        snapshot = mwh_snapshot,

        group_1_wiki_list = ','.join(['"{}"'.format(w) for w in group_1_wikis]),
        group_1_start_timestamp = group_1_ts.strftime('%Y-%m-%d %H:%M:%S'),
        
        group_2_wiki_list = ','.join(['"{}"'.format(w) for w in group_2_wikis]),
        group_2_start_timestamp = group_2_ts.strftime('%Y-%m-%d %H:%M:%S'),

        end_timestamp = end_ts.strftime('%Y-%m-%d %H:%M:%S'),
        known_user_id_expression = make_known_users_sql(known_users, 'wiki_db', 'user_id'),
    )
)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [17]:
## verify first registration in the dataset
user_registrations['user_registration_timestamp'].min()

'2021-05-27 19:16:14.0'

In [18]:
## verify last registration in the dataset
user_registrations['user_registration_timestamp'].max()

'2021-10-13 23:59:39.0'

Those timestamps fit our requirements.

## Get mobile/desktop and API data

This enables us to identify differences between mobile/desktop registrations. We also filter out API registrations as those are almost always mobile apps.

In [21]:
## Get all self-created registrations from ServerSideAccountCreation

ssac_reg_query = '''
SELECT
    wiki AS wiki_db,
    event.userid AS user_id,
    CAST(event.displaymobile AS INT) AS reg_on_mobile,
    event.isapi
FROM event_sanitized.serversideaccountcreation
WHERE {partition_statement}
AND wiki IN ({wiki_list})
AND event.isselfmade = true
'''

In [22]:
ssac_reg_data = spark.run(ssac_reg_query.format(
    wiki_list = ','.join(['"{}"'.format(w) for w in group_1_wikis + group_2_wikis]),
    partition_statement = make_partition_statement(group_1_ts, end_ts)
))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [23]:
## What's the number of registrations?

user_registrations.groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
arwiki,37374
bnwiki,7313
cswiki,4010
fawiki,14704
frwiki,30668
huwiki,1711
plwiki,5247
rowiki,1670
ruwiki,24361


In [24]:
ssac_reg_data.groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
arwiki,37997
bnwiki,7418
cswiki,4087
fawiki,23986
frwiki,50716
huwiki,2867
plwiki,8819
rowiki,2916
ruwiki,39611


In [25]:
## Join with SSAC data

all_users = user_registrations.merge(ssac_reg_data, how = 'left',
                                    on = ['wiki_db', 'user_id'])

In [26]:
## Did we lose anyone?

all_users.loc[all_users['reg_on_mobile'].isna()].groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
arwiki,8
fawiki,1
frwiki,4
rowiki,1
viwiki,2


There's only a handful of users (on the grand scale of things) that we don't appear to have registration data for, so that means that we're good to go. Yay!

We remove users that have `isapi = True`, because those are most likely app registrations.

In [27]:
all_users = all_users.loc[~(all_users['isapi'] == True)]

In [28]:
## Drop the isapi columns

all_users = all_users.drop(columns = ['isapi'])

## Get treatment/control assignments

This involves two operations.

1. Get all users who have the Homepage turned on in their preferences.
2. Get the variant setting for all users.

Users who don't have the Homepage turned on are candidates for the control group, and likewise for the experiment group. Secondary, users are randomly assigned to variants, which we'll use later in the analysis.

In [29]:
def get_prop_settings(wiki, prop, col_name, cast_type, users=None):
    '''
    Query and return a `pandas.DataFrame` with columns `wiki` and `user_id` of all users who have
    the given property turned on in their preferences for that given wiki.
    
    :param wiki: database code of the wiki we're querying
    :type wiki: str
    
    :param prop: the user preference we're querying for
    :type prop: str
    
    :param col_name: name that the column with preference value should have in the
                     resulting DataFrame (e.g. "is_treatment")
    :type col_name: str
    
    :param cast_type: SQL type to cast the property to (in the database this is a BLOB)
    :type cast_type: str
    
    :param users: user IDs of the users we are interested in. This is optional.
    :type users: list
    '''
    
    prop_query = '''
    SELECT "{wiki}" AS wiki, up_user AS user_id,
           CAST(up_value AS {cast_type}) AS {col_name}
    FROM user_properties
    WHERE up_property = "{prop}"
    '''.format(wiki = wiki, prop = prop, cast_type = cast_type, col_name = col_name)
    
    if users is not None:
        prop_query += '''
        AND up_user IN ({})
        '''.format(','.join([str(uid) for uid in users]))
        
    return(mariadb.run(prop_query, wiki))

In [31]:
## Get treatment/control assignments from the MW databases

hp_prefs = pd.concat(
    [get_prop_settings(wiki,
                       'growthexperiments-homepage-enable',
                       'hp_enabled',
                       'UNSIGNED INTEGER') for wiki in group_1_wikis + group_2_wikis]
)

In [32]:
all_users = all_users.merge(hp_prefs, how = 'left',
                            left_on = ['wiki_db', 'user_id'],
                            right_on = ['wiki', 'user_id']).fillna(0)

In [33]:
all_users_agg = (all_users.groupby(['wiki_db', 'hp_enabled'])
                 .agg({'user_id': 'count'})
                 .rename(columns = {'user_id' : 'n_users'}))
all_users_agg['perc'] = (100.0 * all_users_agg['n_users'] / 
                        all_users_agg.groupby('wiki_db')['n_users'].transform('sum'))
all_users_agg

n_users       perc
wiki_db hp_enabled                    
arwiki  0.0            7705  22.417154
        1.0           26666  77.582846
bnwiki  0.0            1585  22.845200
        1.0            5353  77.154800
cswiki  0.0             790  21.572911
        1.0            2872  78.427089
fawiki  0.0            2745  19.965088
        1.0           11004  80.034912
frwiki  0.0            5602  19.979315
        1.0           22437  80.020685
huwiki  0.0             320  20.113136
        1.0            1271  79.886864
plwiki  0.0             953  19.821131
        1.0            3855  80.178869
rowiki  0.0             296  20.598469
        1.0            1141  79.401531
ruwiki  0.0            4103  19.991230
        1.0           16421  80.008770
viwiki  0.0            3325  21.760471
        1.0           11955  78.239529

This is mainly routine, the random assignment hasn't failed in any of our experiments so far. From the percentages above we see that it clearly works on the Group 2 wikis. The Group 1 wikis are all affected by T285996, where during a few days in June 2021 the proportions were switched so 80% of registrations ended up in the Control group rather than the Growth Features group. This is why the Group 1 wikis are all somewhere in the 21.5%–22.5% range.

In [35]:
## Get variant settings from the MW database

variant_prefs = pd.concat(
    [get_prop_settings(wiki,
                       'growthexperiments-homepage-variant',
                       'hp_variant',
                       'CHAR CHARACTER SET utf8') for wiki in group_1_wikis + group_2_wikis]
)

In [36]:
all_users = all_users.merge(variant_prefs, how = 'left',
                           left_on = ['wiki_db', 'user_id'],
                           right_on = ['wiki', 'user_id']).fillna(0)

In [37]:
all_users.groupby(['hp_enabled', 'hp_variant']).agg({'user_id': 'count'})

user_id
hp_enabled hp_variant                 
0.0        0                     27336
           control                  45
           linkrecommendation       43
1.0        0                       132
           control               51479
           linkrecommendation    51364

## Users who turned the Homepage on/off in their preferences

In NEWTEA as well as other experiments we run, we exclude users who turned the Homepage on/off in their preferences because these self-selected in/out of our randomly assigned groups. Our traditional way of doing this was through the PrefUpdate schema, which only has this data for 90 days. In late August 2020, the Growth team deployed a patch that allows us to correlate the user preference setting (`hp_enabled` in the table above) with the variant they're assigned (`hp_variant` above) and thereby exclude users who aren't in a group they should be.

In this case, we can exclude all users in the Control group that have variant set to "control" or "linkrecommendation", and similarly exclude all users in the Growth Features group that do *not* have variant set to "control" or "linkrecommendation".

In [39]:
## Control group users cannot have "control" or "linkrecommendation"
len(all_users.loc[(all_users['hp_enabled'] == 0) &
                  (~all_users['hp_variant'].isin(["control", "linkrecommendation"]))])

27336

In [40]:
## Homepage group users have to have variant "control" or "linkrecommendation"
len(all_users.loc[(all_users['hp_enabled'] == 1) &
                  (all_users['hp_variant'].isin(["control", "linkrecommendation"]))])

102843

In [41]:
all_users_valid = all_users.loc[
    ((all_users['hp_enabled'] == 0) &
     (~all_users['hp_variant'].isin(["control", "linkrecommendation"]))) |
    ((all_users['hp_enabled'] == 1) &
     (all_users['hp_variant'].isin(["control", "linkrecommendation"])))
]

In [42]:
len(all_users_valid)

130179

In [43]:
all_users = all_users_valid

# Dataset Export

Export the dataset to a TSV, both with and without the header for reading into R and the Data Lake.

In [44]:
## Export users and usage data to TSVs for reading into R for analysis

all_users.to_csv(tsv_canonical_user_file, sep = '\t', header = True, index = False)

In [ ]:
all_users.head()

In [45]:
## Write the dataset out into a TSV so it can be imported into the Data Lake.
## NOTE: this file cannot contain a header line.

all_users.to_csv(tsv_datalake_user_file, sep = '\t',
                 columns = ['wiki_db', 'user_id', 'user_registration_timestamp',
                            'reg_on_mobile', 'hp_enabled', 'hp_variant'],
                 header = False, index = False)

In [46]:
## Query to create the canonical user dataset table in Hive

create_table_query = '''
CREATE TABLE {table_name}
(wiki_db STRING COMMENT "wiki this user registered on",
 user_id BIGINT COMMENT "user_id of this user on the given wiki",
 user_registration_timestamp STRING COMMENT "UTC timestamp of user registration",
 reg_on_mobile INT COMMENT "whether the user registered on the mobile site",
 hp_enabled INT COMMENT "whether the user has the Homepage enabled or not",
 hp_variant STRING COMMENT "which variant the user was in"
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY "\t"
LINES TERMINATED BY "\n"
STORED AS TEXTFILE
'''

In [48]:
hive.run(create_table_query.format(
    table_name = canonical_user_table
))

Importing the data was done manually through the command line in order to have a meaningful reference to the actual local file.

In [49]:
import_data_query = '''
LOAD DATA LOCAL INPATH "{local_path}"
OVERWRITE INTO TABLE {table_name}
'''

In [50]:
print(import_data_query.format(
    local_path = tsv_datalake_user_file,
    table_name = canonical_user_table
))


LOAD DATA LOCAL INPATH "/home/nettrom/src/2021-Growth-structured-tasks/datasets/add-a-link-experiment-datalake-users.tsv"
OVERWRITE INTO TABLE nettrom_growth.addalink_exp_users



How many users should we have in the table?

In [51]:
len(all_users)

130179